# 4.2.0 손실함수

### 4.2.1 오차제곱합
---

In [ ]:
import numpy as np

In [4]:
#손실함수-오차제곱합
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [5]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [.1,.05,.6,.0,.05,.1,.0,.1,.0,.0]

sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [7]:
y = [.1,.05,.1,.0,.05,.1,.0,.6,.0,.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

### 4.2.2 교차 엔트로피 오차
---

In [8]:
#손실함수-교차 엔트로피 오차
#delta를 y(신경망의 출력) 값에 더해주는 이유 : log0은 정의 할 수 없을 뿐더러 이론적으로 -무한대에 수렴하기 때문에, 이것을 방지하여 반드시
#0일 수 없도록 아주 작은 상수(1e-7 = 10^-7)을 더해준 것.
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta))

In [9]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [.1,.05,.6,.0,.05,.1,.0,.1,.0,.0]

cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [11]:
y = [.1,.05,.1,.0,.05,.1,.0,.6,.0,.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

### 4.2.3 미니배치 학습
---

In [12]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

In [19]:
#원핫 인코딩으로 정답 위치의 원소만 1이고 나머지는 모두 0인 상태로 불러옴
(x_train, t_train),(x_test,t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) #60000개 데이터와 입력값은 784개가 있다
print(t_train.shape) #60000개 데이터에 대한 정답은 총 10개

(60000, 784)
(60000, 10)


In [20]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)

#무작위로 10개 추출해서 미니배치함
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [21]:
t_batch

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [23]:
np.random.choice(60000,10) 
#0이상 6만 미만의 10개 수를 배열로 리턴시켜줌
#이것을 인덱스로 사용하는 방식

array([28483, 36146, 36255, 51635, 10805,  7009, 18841, 21538,  9589,
       24972])

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기
---

In [24]:
#정답 레이블 y가 원핫 인코딩으로 구성되어 있을 경우 앞서 작성한 교차 엔트로피 오차 함수와 같이 구현할 수 있다.
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y+1e-7))/batch_size

In [25]:
#정답 레이블이 원핫 인코딩이 아니고 2, 7 등 숫자 레이블로 주어졌다면 아래와 같이 교차 엔트로피 오차 함수를 구현할 수 있다
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]+1e-7))/batch_size

# 4.3 수치 미분

### 4.3.1 미분
___

In [26]:
def numerical_diff(f, x):
    h = 10e-50
    return (f(x+h)-f(x))/h